# Example 4: Pipelines

## Set Up

In [1]:
import autompc as ampc
import numpy as np
from autompc.benchmarks import CartpoleSwingupBenchmark

benchmark = CartpoleSwingupBenchmark()


# Get system and task specification
system = benchmark.system
task   = benchmark.task

# Generate benchmark dataset
trajs = benchmark.gen_trajs(seed=100, n_trajs=10, traj_len=200)

Loading AutoMPC...
Cannot import SnoptWrapper
Cannot import KnitroSolver, make sure its Python interface is installed
Finished loading AutoMPC
running build_ext


## Factories
AutoMPC pipelines use Factories to generate models, costs, and controllers from their configurations.

Here is an example for the SINDy model:

In [2]:
from autompc.sysid import SINDyFactory

model_factory = SINDyFactory(system)
model_factory.get_configuration_space()

Configuration space object:
  Hyperparameters:
    lasso_alpha, Type: UniformFloat, Range: [1e-05, 100.0], Default: 1.0, on log-scale
    method, Type: Categorical, Choices: {lstsq, lasso}, Default: lstsq
    poly_basis, Type: Categorical, Choices: {true, false}, Default: false
    poly_cross_terms, Type: Categorical, Choices: {true, false}, Default: false
    poly_degree, Type: UniformInteger, Range: [2, 8], Default: 3
    threshold, Type: UniformFloat, Range: [1e-05, 10.0], Default: 0.01, on log-scale
    time_mode, Type: Categorical, Choices: {discrete, continuous}, Default: discrete
    trig_basis, Type: Categorical, Choices: {true, false}, Default: false
    trig_freq, Type: UniformInteger, Range: [1, 8], Default: 1
    trig_interaction, Type: Categorical, Choices: {true, false}, Default: false
  Conditions:
    lasso_alpha | method in {'lasso'}
    poly_degree | poly_basis in {'true'}
    trig_freq | trig_basis in {'true'}
    trig_interaction | trig_basis in {'true'}

We can use the factory with a given configuration to instantiate (and train the model).

In [3]:
model_cfg = model_factory.get_configuration_space().get_default_configuration()
model_cfg["trig_basis"] = "true"

model = model_factory(model_cfg, trajs)

This works in a similar way for costs and controllers.

In [4]:
from autompc.costs import QuadCostFactory
cost_factory = QuadCostFactory(system)
cost_cfg = cost_factory.get_configuration_space().get_default_configuration()
cost = cost_factory(cost_cfg, task, trajs)

In [5]:
from autompc.control import IterativeLQRFactory
controller_factory = IterativeLQRFactory(system)
controller_cfg = controller_factory.get_configuration_space().get_default_configuration()
controller = controller_factory(controller_cfg, task, trajs)

## Pipelines
We can use pipelines to generate the combined configuration space for a model, cost, and controller.

In [6]:
from autompc.pipeline import Pipeline
pipeline = Pipeline(system, model_factory, cost_factory, controller_factory)
pipeline.get_configuration_space()

Configuration space object:
  Hyperparameters:
    _cost:dx_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:dx_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:omega_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:omega_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:theta_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:theta_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:u_R, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:x_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:x_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _ctrlr:horizon, Type: UniformInteger, Range: [5, 25], Default: 20
    _model:lasso_alpha, Type: UniformFloat, Range: [1e-05, 100.0], Default: 1.0, o

We can then use the pipeline to instantiate all components

In [7]:
pipeline_cfg = pipeline.get_configuration_space().get_default_configuration()
controller, cost, model = pipeline(pipeline_cfg, task, trajs)

It is also easy for us to fix a particular component of the pipeline.  Here is an example of pipeline with a pre-trained model

In [8]:
pipeline_pretrain_model = Pipeline(system, model, cost_factory, controller_factory)
pipeline_pretrain_model.get_configuration_space()

Configuration space object:
  Hyperparameters:
    _cost:dx_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:dx_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:omega_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:omega_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:theta_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:theta_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:u_R, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:x_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:x_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _ctrlr:horizon, Type: UniformInteger, Range: [5, 25], Default: 20